In [1]:
%matplotlib qt
import hyperspy.api as hs

import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

Co = np.loadtxt("Co.txt",comments = "#", delimiter = ",")
CoO = np.loadtxt("CoO.txt",comments = "#", delimiter = ",")
Fe = np.loadtxt("Fe.txt",comments = "#", delimiter = ",")

xCo = Co[:,0]
yCo = Co[:,1]
xCo = np.insert(xCo, 0, 650)
xCo = np.insert(xCo, -1, 850)
yCo = np.insert(yCo,0,0)
yCo = np.insert(yCo,-1,5000)

xCoO = CoO[:,0]
yCoO = CoO[:,1]
xCoO = np.insert(xCoO, 0, 650)
xCoO = np.insert(xCoO, -1, 850)
yCoO = np.insert(yCoO,0,4000)
yCoO = np.insert(yCoO,-1,5000)

xFe = Fe[:,0]
yFe = Fe[:,1]
xFe = np.insert(xFe, 0, 650)
xFe = np.insert(xFe, -1, 850)
yFe = np.insert(yFe,0,0)
yFe = np.insert(yFe,-1,0)

fCo = interpolate.interp1d(xCo,yCo)
fCoO = interpolate.interp1d(xCoO,yCoO)
fFe = interpolate.interp1d(xFe,yFe)

x = np.linspace(650,850, num= 1024)
ybkgdC = 7e12*np.power(x,-3)
ybkgdSh = 3e13*np.power(x,-3.1)
ybkgdCo = 7e13*np.power(x,-3.2)

yCo_interp = fCo(x)
yCoO_interp = fCoO(x)
yFe_interp = fFe(x)

spectre1 = ybkgdC
spectre2 = ybkgdSh + yCoO_interp + yFe_interp
spectre3 = ybkgdCo + yCo_interp


def core_shell (shape_2d, center, radius, ext_radius) :
    xx, yy = np.mgrid[: shape_2d[0], : shape_2d[1]]

    coord = (xx - center[0])**2 + (yy - center[1])**2
    
    circle = coord < radius

    ring_i = coord < ext_radius

    ring = np.logical_xor(circle,ring_i)
    
    return circle.astype(np.float64), ring.astype(np.float64)

shape_2d = [400,400]
centers = [(100,120), (345,302), (80,110), (230, 50), (300,230)]
radii = [300,500,350,600,450]
ext_radii = [500,750,550,900,700]

rings = np.zeros((400,400))
circles = np.zeros((400,400))
for i in range(len(centers)) :
    rings += core_shell(shape_2d,centers[i],radii[i],ext_radii[i])[1]
    circles += core_shell(shape_2d,centers[i],radii[i],ext_radii[i])[0]
    
bot = 1.0 - (circles + rings)

weights = np.zeros((400,400,3))
weights[:,:,0] = bot
weights[:,:,1] = rings
weights[:,:,2] = circles

densities = [100000.0,200000.0,400000.0]

spectra = np.zeros((3,1024))
spectra[0,:] = spectre1/spectre1.sum()*densities[0]
spectra[1,:] = spectre2/spectre2.sum()*densities[1]
spectra[2,:] = spectre3/spectre3.sum()*densities[2]

Y = weights@spectra
spim = hs.signals.Signal1D(Y)
spim.set_signal_type("EELS")

spim.axes_manager[0].units = "nm"
spim.axes_manager[0].name = "x"
spim.axes_manager[0].scale = 0.1
spim.axes_manager[1].units = "nm"
spim.axes_manager[1].name = "y"
spim.axes_manager[1].scale = 0.1
spim.axes_manager[2].scale = 200.0/1024.0
spim.axes_manager[2].offset = 650.0
spim.axes_manager[2].units = "eV"
spim.axes_manager[2].name = "Energy loss"

spim.add_poissonian_noise()
spim.add_gaussian_noise(5.0)

spim.metadata.Acquisition_instrument = {}
spim.metadata.Acquisition_instrument.TEM = {}
spim.metadata.Acquisition_instrument.TEM.Stage = {}
spim.metadata.Acquisition_instrument.TEM.Detector = {}
spim.metadata.Acquisition_instrument.TEM.Detector.EELS = {}
spim.metadata.Acquisition_instrument.TEM.beam_energy = 200
# spim.metadata.Acquisition_instrument.TEM.beam_energy._units = "keV"
spim.metadata.Acquisition_instrument.TEM.beam_current = 0.2
# spim.metadata.Acquisition_instrument.TEM.beam_current._units = "nA"
spim.metadata.Acquisition_instrument.TEM.Detector.EELS.aperture = 5
# spim.metadata.Acquisition_instrument.TEM.Detector.EELS.aperture._units = "mm"
spim.metadata.Acquisition_instrument.TEM.Detector.EELS.dwell_time = 100
spim.metadata.Acquisition_instrument.TEM.Detector.EELS.collection_angle = 22
spim.metadata.Acquisition_instrument.TEM.convergence_angle = 12

spim.metadata.Acquisition_instrument.TEM.Stage.tilt_alpha = 0.0

spim.metadata.Sample = {}
spim.metadata.Sample.elements = ["Co","Fe"]


spim.save("sim_spim_sigma5.hspy")

yCo_sig = hs.signals.Signal1D(yCo_interp)
yCoO_sig = hs.signals.Signal1D(yCoO_interp)
yFe_sig = hs.signals.Signal1D(yFe_interp)

yCo_sig.save("Co.hspy")
yCoO_sig.save("CoO.hspy")
yFe_sig.save("Fe.hspy")

Overwrite 'C:\Users\teurtrie\Travail\QEM\sim_spim_sigma5.hspy' (y/n)?
 n
